## FDW Crop Production Data Profiling - Chad

In [1]:
import os, sys, glob, json
from itertools import product, compress, chain
from functools import reduce
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from tools import save_hdf, save_npz, load_npz, PrintAdminUnits, PlotAdminShapes
from tools import FDW_PD_Sweeper, FDW_PD_AvalTable, FDW_PD_Compiling, FDW_PD_GrainTypeAgg, FDW_PD_ValidateFnidName
from tools import FDW_PD_CreateAdminLink, FDW_PD_RatioAdminLink, FDW_PD_ConnectAdminLink
from tools import FDW_PD_CaliSeasonYear
from tools_graphic import PlotBarProduction, PlotLinePAY, PlotHeatCropSystem, PlotHeatSeasonData
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

In [2]:
# CPCV2 grain code ------------------------------ #
grain_code = pd.read_hdf('./data/crop/grain_cpcv2_code.hdf')
product_category = grain_code[['product', 'product_category']].set_index('product').to_dict()['product_category']
# ----------------------------------------------- #

# Load FEWS NET administrative boundaries ------- #
epsg = 'epsg:32633' # Chad
fn_shapes = sorted(glob.glob('./data/shapefile/fewsnet/TD_Admin?_????.shp'))
shape_all = []
for fn in fn_shapes:
    name = fn[-18:-4]
    exec('%s = gpd.read_file("%s").to_crs("%s")' % (name, fn, epsg))
    exec('%s["area"] = %s["geometry"].area/10**6' % (name, name))
    exec('shape_all.append(%s)' % (name))
shape_all = pd.concat(shape_all, axis=0).reset_index(drop=True)
PrintAdminUnits(shape_all)
# ----------------------------------------------- #

# FDW API host address -------------------------- #
host = 'https://fdw.fews.net'
auth = tuple(json.loads(open('token.json', "r").read()))
parameters = {
    'format': 'json',
    'country': 'Chad',
    'product': 'R011',
    'survey_type': 'crop:best'
}
endpoint = '/api/cropproductionindicatorvalue/'
response = requests.get(host + endpoint, auth=auth, params=parameters, proxies={})
response.raise_for_status()
df = pd.DataFrame.from_records(response.json())
df_origin = df.copy()
# ----------------------------------------------- #

- FEWS NET admin shapefiles ------------------- #
| year	 | Admin1   | # units   | Admin2   | # units   | Admin3   | # units   |
| 1960	 | TD1960A1 | 14	| TD1960A2	| 52	| nan	| 0	|
| 2008	 | TD2008A1 | 22	| TD2008A2	| 62	| nan	| 0	|
| 2012	 | TD2012A1 | 23	| TD2012A2	| 68	| nan	| 0	|
| 2017	 | TD2017A1 | 23	| TD2017A2	| 70	| nan	| 0	|
----------------------------------------------- #


In [3]:
# Manual Pre-processing before Sweeping --------- #
# 1. Default setting 
# a) None-type crop production system
df.loc[df['crop_production_system'].isna(), 'crop_production_system'] = 'none'
df.loc[df['crop_production_system'] == '', 'crop_production_system'] = 'none'
# b) None-type population group
df.loc[df['population_group'].isna(), 'population_group'] = 'none'
df.loc[df['population_group'] == '', 'population_group'] = 'none'
# 2. Manual setting
# a) Add admin names to reporting units
admin_level = 'admin_1'
reporting_unit = df.loc[df['fnid'].map(lambda x: x[6] != 'A'), 'fnid'].unique()
df.loc[df.fnid.isin(reporting_unit),admin_level] = df.loc[df.fnid.isin(reporting_unit), 'locality_name'].map(lambda x: x.split(' ')[0])
# ----------------------------------------------- #

# FDW Production Data Inspection ---------------- #
df, df_raw = FDW_PD_Sweeper(df, area_priority='Area Harvested')
table_dict = FDW_PD_AvalTable(df, shape_all)
# ----------------------------------------------- #

# FEWS NET Shapefile comparison ----------------- #
shape_used = pd.concat([TD_Admin1_1960, TD_Admin1_2008, TD_Admin1_2012], axis=0)
PlotAdminShapes(shape_used, label=True)
# ----------------------------------------------- #

- Remove missing records ---------------------- #
Orignial data points: 5,522
Removed 27 "Missing Value" points
1,786/1,787 "Area Harvested" points are retained.
1,775/1,787 "Quantity Produced" points are retained.
1,934/1,948 "Yield" points are retained.
Current data points: 5,495

- Minor changes are applied ------------------- #

- Basic information --------------------------- #
Data period: 1983 - 2017
6 grain types are found: Fonio, Maize (Corn), Millet, Rice (Paddy), Sorghum, Wheat Grain
2 seasons are found: Main harvest (10-01), Cold off-season harvest (02-01)
1 crop production system are found: none
Data sources include:
[1] Ministry of Agriculture and Irrigation, Chad --- Resultats Definitifs De La Campagne Agricole, Chad
Administrative-1 fnids: 49
Administrative-2 fnids: 0
0 reporting units are found: 

- Total production over time ------------------ #
season_name  Cold off-season harvest Main harvest
                                                 
Fonio                    


- Chad crop seasonal calendar

![FEWS NET](https://fews.net/sites/default/files/styles/large_width_880/public/2023-03/seasonal-calendar-chad.png?itok=IYemQBC5)

- FDW data consists of `TD1960A1`, `TD2008A1`, and `TD2012A1`.

| Year | Admin-1 | # units  | Admin-2  | # units |
| :---: | :----:  | :----:   | :----:   | :---:  |
| 1960 | TD1960A1| 14        | TD1960A2 | 52     |
| 2008 | TD2008A1| 22        | TD2008A2 | 62     |
| 2012 | **`TD2012A1`**| 23 | TD2012A2 | 68    |

- Comparison between admin boundaries.

![image](https://github.com/chc-ucsb/gscd/blob/main/figures/TD_admin_shapes.png?raw=true)

- In 2008, 8 districts are divided and added.

| 1960-2008 |2008-2012|
| :---:|:---:|
|TD1960A124 (Chari-Baguirmi) | TD2008A104 (Chari-Baguirmi), TD2008A108 (Hadjer-Lamis), TD2008A117 (N'Djamena) |
|TD1960A125 (Kanem) | TD2008A109 (Kanem), TD2008A101 (Barh el Gazel)|
|TD1960A126 (Moyen-Chari)| TD2008A116 (Moyen-Chari), TD2008A113 (Mandoul)|
|TD1960A127 (Ouaddai) | TD2008A118 (Ouaddai), TD2008A120 (Sila) | 
|TD1960A128 (B.E.T.) | TD2008A122 (Tibesti), TD2008A103 (Borkou), TD2008A130 (Ennedi)|
|TD1960A129 (Mayo-Kebbi) | TD2008A114 (Mayo-Kebbi Est), TD2008A115 (Mayo-Kebbi Ouest) |

- In 2012, 1 district is divided and added.

| 2008-2012| 2012-present |
| :---:|:---:|
|TD2008A130 (Ennedi) | TD2012A105 (Ennedi-Est), TD2012A106 (Ennedi-Ouest)|

- **`TD2012A1`** is used to represent current admin-level 1 crop data.
- Chad has two crop seasons: `Main harvest` and `Cold off-season`.
- Chad has no population group(s).

In [4]:
# Define the latest shapefile ------------------- #
latest_level = 1
shape_latest = TD_Admin1_2012.copy().to_crs('epsg:4326')
# ----------------------------------------------- #

# Validation of FNIDs and Names ----------------- #
df = FDW_PD_ValidateFnidName(df, shape_used, shape_latest)
df['season_name'] = df['season_name'].replace({'Main harvest':'Main', 'Cold off-season harvest': 'Cold-off'})
# ----------------------------------------------- #

# FDW Production Data Compiling ----------------- #
area, prod = FDW_PD_Compiling(df, shape_used)
area_all, prod_all = area.copy(), prod.copy()
mdx_pss = area.columns.droplevel([0,1]).unique()
# ----------------------------------------------- #

In [5]:
# Link admin boundaries ------------------------- #
link_1960, over_1960 = FDW_PD_CreateAdminLink(TD_Admin1_1960, TD_Admin1_2012, 'ADMIN1', 'ADMIN1', prod, epsg)
link_1960['TD1960A128'].update(method='PBR')
link_1960['TD1960A124'].update(method='PBR')
assert all(np.unique([v['method'] for k,v in link_1960.items()]) == 'PBR')
link_2008, over_2008 = FDW_PD_CreateAdminLink(TD_Admin1_2008, TD_Admin1_2012, 'ADMIN1', 'ADMIN1', prod, epsg)
link_2008['TD2008A130'].update(method='PBR')
link_2008['TD2008A122'].update(method='PBR')
link_2008['TD2008A103'].update(method='PBR')
link_2008['TD2008A108'].update(method='PBR')
link_2008['TD2008A117'].update(method='PBR')
assert all(np.unique([v['method'] for k,v in link_2008.items()]) == 'PBR')
# Crop specific ratios
link_ratio_1960 = FDW_PD_RatioAdminLink(link_1960, prod, over_1960, mdx_pss)
link_ratio_2008 = FDW_PD_RatioAdminLink(link_2008, prod, over_2008, mdx_pss)
# Merge link_ratio
assert link_ratio_1960.keys() == link_ratio_2008.keys()
link_merged = [link_ratio_1960, link_ratio_2008]
fnids_new = list(link_merged[0].keys())
link_ratio = dict()
for fnid in fnids_new:
    container = []
    for link in link_merged:
        container.append(link[fnid])
    link_ratio[fnid] = pd.concat(container, axis=1)
# Add current unit to link_ratio
for fnid_new in link_ratio.keys():
    link_ratio[fnid_new][fnid_new] = 1.0
    link_ratio[fnid_new] = link_ratio[fnid_new].sort_index(axis=1, ascending=False)
# Connect data with AdminLink
area_new, prod_new = FDW_PD_ConnectAdminLink(link_ratio, area, prod, validation=True)
# ----------------------------------------------- #

CBR is considered for 'TD1960A128' as no record found in: ['TD2012A103', 'TD2012A105', 'TD2012A106', 'TD2012A122']
CBR is considered for 'TD1960A124' as no record found in: ['TD2012A117']
CBR is considered for 'TD2008A130' as no record found in: ['TD2012A105', 'TD2012A106']
CBR is considered for 'TD2008A122' as no record found in: ['TD2012A122']
CBR is considered for 'TD2008A103' as no record found in: ['TD2012A103']


In [6]:
# Manual correction ----------------------------- #
crop_new = prod_new/area_new
# ----------------------------------------------- #

# Complete long format DataFrame ---------------- #
df_area = area_new.T.stack().rename('value').reset_index()
df_area['indicator'] = 'area'
df_prod = prod_new.T.stack().rename('value').reset_index()
df_prod['indicator'] = 'production'
df_yield = crop_new.T.stack().rename('value').reset_index()
df_yield['indicator'] = 'yield'
stack = pd.concat([df_area, df_prod, df_yield], axis=0)
# Add "planting year"
cols = ['season_name','product','crop_production_system','planting_month','harvest_year','harvest_month','planting_year']
season_table = df[cols].drop_duplicates()
stack = stack.merge(season_table, on=cols[:-1])
# Add country and admin names
stack = stack.merge(df[['fnid','country','country_code','admin_1','admin_2']].drop_duplicates(), on='fnid', how='inner')
names = [
    'fnid','country','country_code','admin_1','admin_2','name',
    'product','season_name','planting_year','planting_month','harvest_year','harvest_month',
    'crop_production_system','indicator','value'
]
stack_gscd = stack[names]
stack_gscd['gscd_code'] = 'calibrated'
# ----------------------------------------------- #

# Reported FDW data ----------------------------- #
stack_fdw = df[names]
stack_fdw['indicator'] = stack_fdw['indicator'].replace({'Area Harvested':'area','Quantity Produced':'production','Yield':'yield'})
stack_fdw['gscd_code'] = 'reported'
# ----------------------------------------------- #

# Final Processing ------------------------------ #
stack = pd.concat([stack_fdw,stack_gscd], axis=0).reset_index(drop=True)
# No concerns found for grain types
stack['product'] = stack['product'].replace(product_category)
# Calibration of planting and Harvest year and season
cs = {
    'Main': {'planting_month':{'10-01':'06-01'},'harvest_month':{'12-01':'11-01'}},
    'Cold-off': {'planting_month':{'02-01':'01-01'},'harvest_month':{'02-01':'03-01'}},
}
cy = {}
stack, df, link_ratio = FDW_PD_CaliSeasonYear(stack, df, link_ratio, cs, cy)
stack.loc[stack['admin_1'].isna(), 'admin_1'] = 'none'
stack.loc[stack['admin_2'].isna(), 'admin_2'] = 'none'
# ----------------------------------------------- #

# Save data
fn_out = './data/crop/adm_crop_production_TD.csv'
stack.to_csv(fn_out); print(f'{fn_out} is saved.')
save_hdf('./data/crop/adm_crop_production_TD.hdf', stack)
save_hdf('./data/crop/adm_crop_production_TD_raw.hdf', df)
save_npz('./data/crop/adm_crop_production_TD_ratio.npz', link_ratio)

./data/crop/adm_crop_production_TD.csv is saved.
./data/crop/adm_crop_production_TD.hdf is saved.
./data/crop/adm_crop_production_TD_raw.hdf is saved.
./data/crop/adm_crop_production_TD_ratio.npz is saved.


## Visualization of production data

In [7]:
# Bar chart of national grain production
country_iso, country_name = 'TD', 'Chad'
df = pd.read_hdf('./data/crop/adm_crop_production_%s.hdf' % country_iso)
df = df[df['gscd_code']=='calibrated']; df['year'] = df['harvest_year']
year = [df['year'].min(), df['year'].max()]
product_order = ['Sorghum','Millet','Maize','Rice','Fonio','Wheat']
for season_name in ['Main','Cold-off']:
    footnote = 'National grain production in %s - %s' % (country_name, season_name)
    fn_save = './figures/%s_bar_natgrainprod_%s.png' % (country_iso, season_name)
    sub = df[df['season_name'] == season_name]
    fig = PlotBarProduction(sub, year, product_order, footnote, fn_save)
    # fig.show()

./figures/TD_bar_natgrainprod_Main.png is saved.
./figures/TD_bar_natgrainprod_Cold-off.png is saved.


![image](https://github.com/chc-ucsb/gscd/blob/main/figures/TD_bar_natgrainprod_Main.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/TD_bar_natgrainprod_Cold-off.png?raw=true)

In [8]:
# Lineplot of Production-Area-Yield (PAY) time-series
country_iso, country_name = 'TD', 'Chad'
df = pd.read_hdf('./data/crop/adm_crop_production_%s.hdf' % country_iso)
df = df[df['gscd_code']=='calibrated']; df['year'] = df['harvest_year']
year = [df['year'].min(), df['year'].max()]
product_season = [
    ['Sorghum','Main'],
    ['Millet','Main'],
    ['Maize','Main'],
    ['Sorghum','Cold-off'],
]
for product_name, season_name in product_season:
    footnote = 'Production-Area-Yield (PAY) time-series of %s - %s - %s' % (country_iso, product_name, season_name)
    fn_save = './figures/%s_line_pay_%s_%s.png' % (country_iso, product_name, season_name)
    sub = df[(df['product'] == product_name) & (df['season_name'] == season_name)]
    fig = PlotLinePAY(sub, year, footnote, fn_save)
    # fig.show()

./figures/TD_line_pay_Sorghum_Main.png is saved.
./figures/TD_line_pay_Millet_Main.png is saved.
./figures/TD_line_pay_Maize_Main.png is saved.
./figures/TD_line_pay_Sorghum_Cold-off.png is saved.


![image](https://github.com/chc-ucsb/gscd/blob/main/figures/TD_line_pay_Sorghum_Main.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/TD_line_pay_Millet_Main.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/TD_line_pay_Maize_Main.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/TD_line_pay_Sorghum_Cold-off.png?raw=true)

In [9]:
# Heatmap of seasonal data availability
country_iso, country_name = 'TD', 'Chad'
df = pd.read_hdf('./data/crop/adm_crop_production_%s_raw.hdf' % country_iso)
df['year'] = df['harvest_year']
code = {'Main':1, 'Cold-off':2}
comb = {1:1,10:2,11:3}
comb_name = {1:'Main',2:'Cold-off',3:'Main + Cold-off'}
for product_name in ['Sorghum','Millet','Maize (Corn)']:
    data = df[(df['product'] == product_name) & (df['season_name'].isin(code.keys()))]
    footnote = 'Seasonal data availability in %s - %s (uncorrected)' % (country_name, product_name)
    fn_save = './figures/%s_heat_seasondata_%s.png' % (country_iso, product_name)
    fig = PlotHeatSeasonData(data, code, comb, comb_name, footnote, fn_save)
    # fig.show()

./figures/TD_heat_seasondata_Sorghum.png is saved.
./figures/TD_heat_seasondata_Millet.png is saved.
./figures/TD_heat_seasondata_Maize (Corn).png is saved.


![image](https://github.com/chc-ucsb/gscd/blob/main/figures/TD_heat_seasondata_Sorghum.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/TD_heat_seasondata_Millet.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/TD_heat_seasondata_Maize%20(Corn).png?raw=true)

In [10]:
# Calibrated PAY time-series per FNID
from tools_graphic import PlotLineCropTS
country_iso, country_name = 'TD', 'Chad'
df = pd.read_hdf('./data/crop/adm_crop_production_%s.hdf' % country_iso)
df = df[df['gscd_code']=='calibrated']; df['year'] = df['harvest_year']
link_ratio = load_npz('./data/crop/adm_crop_production_%s_ratio.npz' % country_iso)
year_all = np.arange(df['year'].min(), df['year'].max()+1)
prod_season = [
    ['Sorghum','Main'],
    ['Millet','Main'],
    ['Maize','Main'],
    ['Sorghum','Cold-off'],    
]
for product_name, season_name in prod_season:
    sub = df[
        (df['product'] == product_name) &
        (df['season_name'] == season_name)
    ]
    for fnid in sub['fnid'].unique():
        sub_fps = sub[sub['fnid'] == fnid]
        fn_save = './figures/crop_calibrated/%s_%s_%s_%s.png' % (country_iso, product_name, season_name, fnid)
        fig = PlotLineCropTS(sub_fps, fnid, product_name, season_name, link_ratio, year_all, fn_save)

./figures/crop_calibrated/TD_Sorghum_Main_TD2012A101.png is saved.
./figures/crop_calibrated/TD_Sorghum_Main_TD2012A102.png is saved.
./figures/crop_calibrated/TD_Sorghum_Main_TD2012A104.png is saved.
./figures/crop_calibrated/TD_Sorghum_Main_TD2012A107.png is saved.
./figures/crop_calibrated/TD_Sorghum_Main_TD2012A108.png is saved.
./figures/crop_calibrated/TD_Sorghum_Main_TD2012A111.png is saved.
./figures/crop_calibrated/TD_Sorghum_Main_TD2012A112.png is saved.
./figures/crop_calibrated/TD_Sorghum_Main_TD2012A113.png is saved.
./figures/crop_calibrated/TD_Sorghum_Main_TD2012A114.png is saved.
./figures/crop_calibrated/TD_Sorghum_Main_TD2012A115.png is saved.
./figures/crop_calibrated/TD_Sorghum_Main_TD2012A116.png is saved.
./figures/crop_calibrated/TD_Sorghum_Main_TD2012A118.png is saved.
./figures/crop_calibrated/TD_Sorghum_Main_TD2012A119.png is saved.
./figures/crop_calibrated/TD_Sorghum_Main_TD2012A120.png is saved.
./figures/crop_calibrated/TD_Sorghum_Main_TD2012A121.png is sa